# Fluent Python
https://github.com/fluentpython/example-code

파이썬 용어집 

https://docs.python.org/ko/3/glossary.html

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np

def time_check(func):
    def decorated():
        import timea
        start = time.time()
        func()
        print("---{}s seconds---".format(time.time()-start_time))
    return decorated# Fluent Python

## CHAPTER 14. Iterables, Iterators, and Generators

데이터셋을 검색할 때는 *느긋하게* 가져와야한다. 즉, 한번에 하나씩 그리고 필요할때 가져와야 한다. 

반복형 
- for 루프 
- 컬렉션형 생성과 확장
- 텍스트 파일을 한줄씩 반복
- 지능형 리스트, 딕셔너리, 집합
- 튜플 언패킹
- 함수 호출 시 * 를 이용해서 실제 매개변수를 언패킹

### Sentence Take #1: A Sequence of Words

#### Why Sequences Are Iterable: The iter Function

iter() 함수 steps
1. 객체가 \_\_iter\_\_() 메서드를 구현하는지 확인, 반복자를 가져옴
2. \_\_iter\_\_() 없어도 \_\_getitem\_\_() 이 구현되어 있다면, 파이썬은 인텍스 0 에서 시작해서 항목을 순서대로 가져오는 반복자를 생성
3. 전부 구현이 안되어 있으면, TypeError 'C' object is not iterable 메시지와 에러가 발생. 'C' 는 대상객체 클래스


### Iterables Versus Iterators

**iterable**
- iter() 내장 함수가 반복자를 가져올 수 있는 모든 객체와 반복자를 반환하는 \_\_iter\_\_() 메서드 구현 객체는 iterable 이다. \_\_getitem\_\_() 구현 객체인 스퀀스도 마찬가지다.

**\_\_next\_\_()**
- 다음에 사용할 항목을 반환. 더이상 항목이 없으면 StopIteration 을 발생

**\_\_iter\_\_()**
- self 반환. 루프문등 필요한곳에서 iterator 를 사용 할수 있게함


A concrete Iterable.iter should return a new Iterator instance. A concrete Iterator must implement next. The Iterator.iter method just returns the instance itself.

The Iterator ABC implements __iter__ by doing return self.



**Lib/types.py**
- 파인썬의 반복자는 자료형이 아니라 프로토콜이다.
- 상당히 많은 유동적인 수의 내장 자료형이 반복자의 *일부* 를 구현다. 
- 자료형을 검사하면 안된다. 대신 hasattr() 를 이용해서 "\_\_iter\_\_()" 와 "\_\_next\_\_()" 속성이 있는지 검사하라

이런 조언과 \_collections\_adb.py 에 구현된 논리를 고려하면 x 가 반복자 인지 확인 하는 가장 좋은 방법은 isinstance(x, abc.iterator) 를 호출하는 것이다. Iterator.\_\_subclasshook\_\_() 메서드 덕분에 이 방법은 x 가 Iterator 의 실제 서브클래스 인 경우와 가상 서브클래스인 경우 모두 제대로 작동한다. 

반복자 필수 메서드 \_\_next\_\_(), \_\_iter\_\_() 밖에 없다. 반복자는 재설정 할수 없다. this will not reset a depleted iterator.

**iterator**
* 다음 항목을 반환하거나, 다음 항목이 없을 때 StopIteration 예외를 발생시키는, 인수를 받지 않는 \_\_next\_\_() 메서드를 구현하는 객체, 

### Sentence Take #2: A Classic Iterator

In [3]:
import re
import reprlib
RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self): # 반복자를 리턴함 (반복형 프로토콜 구현)
        return SentenceIterator(self.words)

class SentenceIterator:
    def __init__(self, words):
        self.words = words
        self.index = 0
        
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word
    
    def __iter__(self):
        return self

* 반복형은 호출될 때마다 본복자를 새로 생성하는 \_\_iter\_\_() 메서드를 가지고 있다. 
* 반복자는 개별항목을 반환하는 \_\_next\_\_() 메서드와 self 를 반환하는 \_\_iter\_\_() 메서드를 가진다. 

반복자 패턴 용도
* 집합 객체의 내부 표현을 노출시키지 않고 내용에 접근하는 경우
* 집합 객체의 다중 반복을 지원하는 경우
* 다양한 집합 구조체를 반복하기 위한 통일된 인터페이스를 제공하는 경우 

tip) 
반복형은 자기가신을 반복하는 반복자로 작동하면 안된다. <br>
반복자는 반복형이 되어야 한다. 이떄 반복자의 \_\_iter\_\_() 는 self 를 반환해야 한다. 

### Sentence Take #3: A Generator Function

In [4]:
import re
import reprlib
RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for word in self.words:
            yield word
        return
# done! 별도의 반복자 클래스가 필요 없다. 

제너레이터 함수는 함수 본체를 포함하는 제너레이터 객체를 생성한다. 

제너레이터는 값을 **생성** 한다. 함수는 값을 반환하지만 제너레이터 함수는 제너레이터 객체가 반환된다. 

### Sentence Take #4: A Lazy Implementation

In [5]:
import re
import reprlib
RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for match in RE_WORD.finditer(self.text):
            yield match.group()

Iterator 인터페이스는 느긋하게 처리하도록 설계되어 있다. next(my_iterator) 는 한번에 한 항목만 생성한다. 

**lazy evaluation** and **eager evaluation** are actual technical terms in programming language theory.

re.finditer() 함수는 re.findall() 의 느긋한 버전으로, 리스트 대신 필요에 따라 re.MatchObject 객체를 생성하는 제너레이터를 반환한다. 매칭되는 항목이 많으면 re.finditer() 가 메모리를 많이 절약해준다. 

### Sentence Take #5: A Generator Expression

In [6]:
import re
import reprlib
RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self): #제너레이터 표현식 사용
        return (match.group() for match in RE_WORD.finditer(self.text)) 

### Another Example: Arithmetic Progression Generator

In [7]:
class ArithmeticProgression:
    
    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step
        self.end = end # None -> "infinite" series

    def __iter__(self):
        result = type(self.begin + self.step)(self.begin)
        # 가독성 높이기 위해 self.end 가 None 이면 forever 을 True 로 설정 (무한등차수열)
        forever = self.end is None  
        index = 0
        while forever or result < self.end:
            yield result
            index += 1
            result = self.begin + self.step * index

In [8]:
ap = ArithmeticProgression(0, 1, 3)
list(ap)

[0, 1, 2]

In [11]:
from fractions import Fraction
ap = ArithmeticProgression(0, Fraction(1, 3), 1)
list(ap)

[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]

In [10]:
from decimal import Decimal
ap = ArithmeticProgression(0, Decimal('.1'), .3)
list(ap)

[Decimal('0'), Decimal('0.1'), Decimal('0.2')]

### Arithmetic Progression with itertools

The itertools module in Python 3.4 has 19 generator functions that can be combined in a variety of interesting ways.

itertools.count() : 숫자를 생성하는 제너레이터를 반환, 끝이 없다. list(count()) 를 실행하면, 메모리보다 큰 리스트를 만들려고 시도하다 실패한다. 

itertools.takewhile() : 다른 제너레이터를 소비하면서 predicate (주어진 조건식) 이 False 가 되면 중단되는 제너레이터를 생성한다. 

In [14]:
import itertools
gen = itertools.takewhile(lambda n: n < 3, itertools.count(1, .5))
list(gen)

[1, 1.5, 2.0, 2.5]

### Generator Functions in the Standard Library

#### <필터링 제너레이터함수>

**itertools.ompress(it, selector_it)** : Consumes two iterables in parallel; yields items from it whenever the corresponding item in selector_it is truthy

**itertools.dropwhile(predicate, it)** : Consumes it skipping items while predicate computes truthy, then yields very remaining item (no further checks are made)

**filter(predicate,it)** : Applies predicate to each item of iterable, yielding the item if predi
cate(item) is truthy; if predicate is None, only truthy items are yielded

**itertools.filterfalse(predicate, it)** : Same as filter, with the predicate logic negated: yields items whenever predicate computes falsy

**itertools.islice(it, stop) or islice(it,start, stop,step=1)** : Yields items from a slice of it, similar to s[:stop] or s[start:stop:step] except it can be any iterable, and the operation is lazy

**itertools.takewhile(predicate, it)** : Yields items while predicate computes truthy, then stops and no further checks are made

In [15]:
def vowel(c):
    return c.lower() in 'aeiou'

list(filter(vowel, 'Aardvark'))

import itertools
list(itertools.filterfalse(vowel, 'Aardvark'))

list(itertools.dropwhile(vowel, 'Aardvark'))

list(itertools.takewhile(vowel, 'Aardvark'))

list(itertools.compress('Aardvark', (1,0,1,1,0,1)))

list(itertools.islice('Aardvark', 4))

list(itertools.islice('Aardvark', 4, 7))

list(itertools.islice('Aardvark', 1, 7, 2))

['A', 'a', 'a']

['r', 'd', 'v', 'r', 'k']

['r', 'd', 'v', 'a', 'r', 'k']

['A', 'a']

['A', 'r', 'd', 'a']

['A', 'a', 'r', 'd']

['v', 'a', 'r']

['a', 'd', 'a']

#### < Mapping generator functions >

**itertools.accumulate(it, [func])** : Yields accumulated sums; if func is provided, yields the result of applying it to the first pair of items, then to the first result and next item, etc.

**enumerate(iterable, start=0)** : Yields 2-tuples of the form (index, item), where index is counted from
start, and item is taken from the iterable

**map(func, it1,[it2, …, itN])** : Applies func to each item of it, yielding the result; if N iterables are given, func must take N arguments and the iterables will be consumed in parallel

**itertools.starmap(func, it)** Applies func to each item of it, yielding the result; the input iterable should yield iterable items iit, and func is applied as func(*iit)

In [16]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]

import itertools
list(itertools.accumulate(sample)) #

list(itertools.accumulate(sample, min)) #

list(itertools.accumulate(sample, max)) #

import operator
list(itertools.accumulate(sample, operator.mul)) #

list(itertools.accumulate(range(1, 11), operator.mul))


[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]

[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

In [19]:
list(enumerate('albatroz', 1)) #

import operator
list(map(operator.mul, range(11), range(11))) #

list(map(operator.mul, range(11), [2, 4, 8])) #

list(map(lambda a, b: (a, b), range(11), [2, 4, 8])) #

import itertools
list(itertools.starmap(operator.mul, enumerate('albatroz', 1))) #

sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
list(itertools.starmap(lambda a, b: b/a, enumerate(itertools.accumulate(sample), 1)))

[(1, 'a'),
 (2, 'l'),
 (3, 'b'),
 (4, 'a'),
 (5, 't'),
 (6, 'r'),
 (7, 'o'),
 (8, 'z')]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

[0, 4, 16]

[(0, 2), (1, 4), (2, 8)]

['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']

[5.0,
 4.5,
 3.6666666666666665,
 4.75,
 5.2,
 5.333333333333333,
 5.0,
 4.375,
 4.888888888888889,
 4.5]

#### < Generator functions that merge multiple input iterables >

**itertools.chain(it1, …, itN)** Yield all items from it1, then from it2 etc., seamlessly

**itertools.chain.from_iterable(it)** Yield all items from each iterable produced by it, one after the other, seamlessly; it should yield iterable items, for example, a list of iterables

**itertools product(it1, …, itN, repeat=1)** Cartesian product: yields N-tuples made by combining items from each input iterable like nested for loops could produce; repeat allows the input iterables to be consumed more than once

**zip(it1, …, itN)** Yields N-tuples built from items taken from the iterables in parallel, silently
stopping when the first iterable is exhausted 

**itertools zip_longest(it1, …,itN, fillvalue=None)** Yields N-tuples built from items taken from the iterables in parallel, stopping only when

In [20]:
list(itertools.chain('ABC', range(2))) #

list(itertools.chain(enumerate('ABC'))) #

list(itertools.chain.from_iterable(enumerate('ABC'))) #

list(zip('ABC', range(5))) #

list(zip('ABC', range(5), [10, 20, 30, 40])) #

list(itertools.zip_longest('ABC', range(5))) #

list(itertools.zip_longest('ABC', range(5), fillvalue='?'))

['A', 'B', 'C', 0, 1]

[(0, 'A'), (1, 'B'), (2, 'C')]

[0, 'A', 1, 'B', 2, 'C']

[('A', 0), ('B', 1), ('C', 2)]

[('A', 0, 10), ('B', 1, 20), ('C', 2, 30)]

[('A', 0), ('B', 1), ('C', 2), (None, 3), (None, 4)]

[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]

In [22]:
list(itertools.product('ABC', range(2))) #

suits = 'spades hearts diamonds clubs'.split()

list(itertools.product('AK', suits)) #

list(itertools.product('ABC')) #

list(itertools.product('ABC', repeat=2)) #

list(itertools.product(range(2), repeat=3))

rows = itertools.product('AB', range(2), repeat=2)
for row in rows: print(row)

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]

[('A', 'spades'),
 ('A', 'hearts'),
 ('A', 'diamonds'),
 ('A', 'clubs'),
 ('K', 'spades'),
 ('K', 'hearts'),
 ('K', 'diamonds'),
 ('K', 'clubs')]

[('A',), ('B',), ('C',)]

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

('A', 0, 'A', 0)
('A', 0, 'A', 1)
('A', 0, 'B', 0)
('A', 0, 'B', 1)
('A', 1, 'A', 0)
('A', 1, 'A', 1)
('A', 1, 'B', 0)
('A', 1, 'B', 1)
('B', 0, 'A', 0)
('B', 0, 'A', 1)
('B', 0, 'B', 0)
('B', 0, 'B', 1)
('B', 1, 'A', 0)
('B', 1, 'A', 1)
('B', 1, 'B', 0)
('B', 1, 'B', 1)


#### < Generator functions that expand each input item into multiple output items Module Function >

**itertools.combinations(it, out_len)** Yield combinations of out_len items from the items yielded by it

**itertools.combinations_with_replacement(it, out_len)** Yield combinations of out_len items from the items yielded by it, including combinations with repeated items

**itertools.count(start=0, step=1)** Yields numbers starting at start, incremented by step, indefinitely

**itertools.cycle(it)** Yields items from it storing a copy of each, then yields the entire
sequence repeatedly, indefinitely

**itertools.permutations(it,out_len=None)** Yield permutations of out_len items from the items yielded by it; by default, out_len is len(list(it))

**itertools.repeat(item, [times])** Yield the given item repeadedly, indefinetly unless a number of times is given

In [23]:
ct = itertools.count() #
next(ct) #

next(ct), next(ct), next(ct) #

list(itertools.islice(itertools.count(1, .3), 3)) #

cy = itertools.cycle('ABC') #
next(cy)

list(itertools.islice(cy, 7)) #

rp = itertools.repeat(7) #
next(rp), next(rp)

list(itertools.repeat(8, 4)) #

list(map(operator.mul, range(11), itertools.repeat(5)))

0

(1, 2, 3)

[1, 1.3, 1.6]

'A'

['B', 'C', 'A', 'B', 'C', 'A', 'B']

(7, 7)

[8, 8, 8, 8]

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

In [24]:
list(itertools.combinations('ABC', 2)) #

list(itertools.combinations_with_replacement('ABC', 2)) #

list(itertools.permutations('ABC', 2)) #

list(itertools.product('ABC', repeat=2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

#### < Rearranging generator functions >

**itertools groupby(it,key=None)** Yields 2-tuples of the form (key, group), where key is the grouping criterion and group is a generator yielding the items in the group

**reversed(seq)** Yields items from seq in reverse order, from last to first; seq must be a sequence
or implement the __reversed__ special method

**itertools tee(it, n=2)** Yields a tuple of n generators, each yielding the items of the input iterable
independently

In [25]:
list(itertools.groupby('LLLLAAGGG'))

for char, group in itertools.groupby('LLLLAAAGG'): #
    print(char, '->', list(group))
    
animals = ['duck', 'eagle', 'rat', 'giraffe', 'bear', 'bat', 'dolphin', 'shark', 'lion']

animals.sort(key=len)
animals

for length, group in itertools.groupby(animals, len): #
    print(length, '->', list(group))
    
for length, group in itertools.groupby(reversed(animals), len): #
    print(length, '->', list(group))

[('L', <itertools._grouper at 0x7f8451401898>),
 ('A', <itertools._grouper at 0x7f8451401cf8>),
 ('G', <itertools._grouper at 0x7f84514019e8>)]

L -> ['L', 'L', 'L', 'L']
A -> ['A', 'A', 'A']
G -> ['G', 'G']


['rat', 'bat', 'duck', 'bear', 'lion', 'eagle', 'shark', 'giraffe', 'dolphin']

3 -> ['rat', 'bat']
4 -> ['duck', 'bear', 'lion']
5 -> ['eagle', 'shark']
7 -> ['giraffe', 'dolphin']
7 -> ['dolphin', 'giraffe']
5 -> ['shark', 'eagle']
4 -> ['lion', 'bear', 'duck']
3 -> ['bat', 'rat']


In [26]:
list(itertools.tee('ABC'))

g1, g2 = itertools.tee('ABC')

next(g1)

next(g2)

next(g2)

list(g1)

list(g2)

list(zip(*itertools.tee('ABC')))

[<itertools._tee at 0x7f8451450c48>, <itertools._tee at 0x7f8451450bc8>]

'A'

'A'

'B'

['B', 'C']

['C']

[('A', 'A'), ('B', 'B'), ('C', 'C')]

### New Syntax in Python 3.3: yield from


In [27]:
def chain(*iterables):
    for it in iterables:
        for i in it:
            yield i
            
s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

In [28]:
def chain(*iterables):
    for i in iterables:
        yield from i

list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

### Iterable Reducing Functions

**all(it)** Returns True if all items in it are truthy, otherwise False; all([]) returns True

**any(it)** Returns True if any item in it is truthy, otherwise False; any([]) returns False

**max(it, [key=,] [default=])** Returns the maximum value of the items in it;a key is an ordering
function, as in sorted; default is returned if the iterable is empty 

**min(it, [key=,] [default=])** Returns the minimum value of the items in it.b key is an ordering function, as in sorted; default is returned if the iterable is empty 

**functools reduce(func, it, [initial])** Returns the result of applying func to the first pair of items, then to that result and the third item and so on; if given, initial forms the initial pair with the first item

**sum(it, start=0)** The sum of all items in it, with the optional start value

In [29]:
all([1, 2, 3])

all([1, 0, 3])

all([])

any([1, 2, 3])

any([1, 0, 3])

any([0, 0.0])

any([])

g = (n for n in [0, 0.0, 7, 8])
any(g)

next(g)

True

False

True

True

True

False

False

True

8

### A Closer Look at the iter Function

첫번째 인수는 값을 생성하기 위해 인수 없이 반복 적으로 호출되는 콜러블이여야 하고

두번째 인수는 구분 표시 sentinel로 콜러블에서 이 값이 반복되며 반복자가 StopIteration 예외를 발생시키도록 한다.

In [4]:
from random import randint 

def d6():
    return randint(1, 6)

In [5]:
d6_iter = iter(d6, 1)
d6_iter

In [6]:
for roll in d6_iter:
    print(roll)

4


```python
with open('mydata.txt') as fp:
    for line in iter(fp.readline, ''):
        process_line(line)
```